In [2]:
import os
import threading
import numpy as np

# Flask and web-related imports
from flask import Flask, render_template, request
from IPython.display import display, HTML

# TensorFlow and Keras imports
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


2025-06-15 10:25:26.857499: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-15 10:25:26.861831: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-15 10:25:26.871290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749979526.888522   96788 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749979526.893291   96788 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749979526.907409   96788 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
# Create a 'templates' folder if it doesn't exist
if not os.path.exists('templates'):
    os.makedirs('templates')

In [4]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analyzer</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; background-color: #f4f4f9; color: #333; max-width: 600px; margin: 40px auto; padding: 20px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        h1 { color: #444; text-align: center; }
        form { display: flex; flex-direction: column; }
        textarea { padding: 10px; border-radius: 4px; border: 1px solid #ddd; margin-bottom: 10px; font-size: 1rem; resize: vertical; }
        input[type="submit"] { background-color: #007bff; color: white; padding: 12px; border: none; border-radius: 4px; cursor: pointer; font-size: 1rem; font-weight: bold; }
        input[type="submit"]:hover { background-color: #0056b3; }
        .result { margin-top: 20px; padding: 15px; border-radius: 4px; }
        .result.positive { background-color: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
        .result.negative { background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
        .result p { margin: 0; }
        .result span { font-weight: bold; }
    </style>
</head>
<body>
    <h1>💬 Sentiment Analyzer</h1>
    <form method="post">
        <textarea name="review_text" rows="4" placeholder="Enter a movie review..."></textarea>
        <input type="submit" value="Analyze Sentiment">
    </form>

    {% if result %}
        <div class="result {{ result.sentiment.lower() }}">
            <p>Your Review: <span>"{{ result.text }}"</span></p>
            <p>Predicted Sentiment: <span>{{ result.sentiment }}</span></p>
            <p>Confidence: <span>{{ '%.2f'|format(result.confidence * 100) }}%</span></p>
        </div>
    {% endif %}
</body>
</html>

Writing templates/index.html


In [5]:
# Cell 5: The Flask App Logic (Corrected)

# --- Global Variables & Model Loading (Done Once) ---
app = Flask(__name__)
VOCAB_SIZE = 10000
MAX_LEN = 200

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=MAX_LEN)
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

print("Loading word index...")
WORD_INDEX = imdb.get_word_index()

# Define the model architecture
def build_model():
    model = Sequential([
        Embedding(input_dim=VOCAB_SIZE, output_dim=32),
        SimpleRNN(units=32),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create the model instance
model = build_model()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.3)#
model.summary() # O see the built model structure


# --- Preprocessing Function ---
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [1] # Start with <SOS>
    for word in words:
        index = WORD_INDEX.get(word, 2) + 3 # +3 offset, default to <UNK>
        encoded_review.append(index if index < VOCAB_SIZE else 2)
    return pad_sequences([encoded_review], maxlen=MAX_LEN, padding='pre', truncating='pre')


# --- Flask Routes ---
@app.route('/', methods=['GET', 'POST'])
def home():
    result = None
    if request.method == 'POST':
        user_text = request.form.get('review_text')
        if user_text:
            processed_input = preprocess_text(user_text)
            prediction_prob = model.predict(processed_input)[0][0]
            if prediction_prob > 0.5:
                sentiment = "Positive 👍😁"
            else:
                sentiment = "Negative 😒"
            # sentiment = "Positive 👍😁" if prediction_prob > 0.5 else "Negative 😒"
            confidence = prediction_prob if sentiment == "Positive" else 1 - prediction_prob
            result = {"text": user_text, "sentiment": sentiment, "confidence": confidence}
    return render_template('index.html', result=result)


# --- Run the App in a Thread ---
# This starts the server without blocking the notebook
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5002, use_reloader=False)).start()


# --- Display a Clickable Link ---
# This makes it easy to open the web interface
display(HTML('<a href="http://localhost:5002" target="_blank">Click to Open Web Interface</a>'))

Loading word index...
Epoch 1/5


2025-06-15 10:25:33.951650: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


274/274 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.5323 - loss: 0.6910 - val_accuracy: 0.5331 - val_loss: 0.6889
Epoch 2/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.5679 - loss: 0.6783 - val_accuracy: 0.5695 - val_loss: 0.6760
Epoch 3/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6377 - loss: 0.6433 - val_accuracy: 0.5619 - val_loss: 0.6767
Epoch 4/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6305 - loss: 0.6409 - val_accuracy: 0.5777 - val_loss: 0.7474
Epoch 5/5
274/274 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.6843 - loss: 0.5998 - val_accuracy: 0.6099 - val_loss: 0.6589


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Optimizer params: 644,228 (2.46 MB)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://10.0.139.82:5002
Press CTRL+C to quit
127.0.0.1 - - [15/Jun/2025 10:26:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2025 10:26:18] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


127.0.0.1 - - [15/Jun/2025 10:26:28] "POST / HTTP/1.1" 200 -
